In [1]:
!pip install keras
!pip install tensorflow
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install sklearn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline



In [3]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.callbacks import ReduceLROnPlateau
np.random.seed(1000)

Using TensorFlow backend.


In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 



In [6]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [7]:
X_train = X_train / 255.0
test = test/ 255.0


In [8]:
#Reshaping the data
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

print(X_train.shape)
print(test.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [9]:
#converting to categorical 
Y_train= to_categorical(Y_train, num_classes=10)

print(Y_train.shape)


(42000, 10)


In [10]:

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.25, random_state=2)

In [11]:
model = Sequential()


model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
# The input of this layer is the output of the first layer, which is a 28 * 28 * 6 node matrix.
# The size of the filter used in this layer is 2 * 2, and the step length and width are both 2, so the output matrix size of this layer is 14 * 14 * 6.
#Batch Normalization
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.16))

# The input matrix size of this layer is 14 * 14 * 6, the filter size used is 5 * 5, and the depth is 16. This layer does not use all 0 padding, and the step size is 1.
# The output matrix size of this layer is 10 * 10 * 16. This layer has 5 * 5 * 6 * 16 + 16 = 2416 parameters
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
# The input matrix size of this layer is 10 * 10 * 16. The size of the filter used in this layer is 2 * 2, and the length and width steps are both 2, so the output matrix size of this layer is 5 * 5 * 16.
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.26))
# The input matrix size of this layer is 5 * 5 * 16. This layer is called a convolution layer in the LeNet-5 paper, but because the size of the filter is 5 * 5, #
# So it is not different from the fully connected layer. If the nodes in the 5 * 5 * 16 matrix are pulled into a vector, then this layer is the same as the fully connected layer.
# The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
model.add(Flatten())
model.add(Dense(120, activation='relu'))
# The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
model.add(Dense(84, activation='relu'))
# The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 6)         24        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 6)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 16)         

In [12]:
model.fit(X_train, Y_train, batch_size=128, epochs=25, verbose=1, validation_data=(X_val, Y_val))
score = model.evaluate(X_val, Y_val)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Train on 31500 samples, validate on 10500 samples
Epoch 1/25
31500/31500 [==============================] - 7s 234us/step - loss: 0.5538 - accuracy: 0.8220 - val_loss: 0.4579 - val_accuracy: 0.9412
Epoch 2/25
31500/31500 [==============================] - 9s 277us/step - loss: 0.1682 - accuracy: 0.9458 - val_loss: 0.1201 - val_accuracy: 0.9679
Epoch 3/25
31500/31500 [==============================] - 9s 277us/step - loss: 0.1283 - accuracy: 0.9591 - val_loss: 0.0777 - val_accuracy: 0.9761
Epoch 4/25
31500/31500 [==============================] - 9s 278us/step - loss: 0.1042 - accuracy: 0.9667 - val_loss: 0.0684 - val_accuracy: 0.9779
Epoch 5/25
31500/31500 [==============================] - 9s 278us/step - loss: 0.0948 - accuracy: 0.9702 - val_loss: 0.0564 - val_accuracy: 0.9827
Epoch 6/25
31500/31500 [==============================] - 9s 278us/step - loss: 0.0792 - accuracy: 0.9742 - val_loss: 0.0543 - val_accuracy: 0.9841
Epoch 7/25
31500/31500 [==============================] - 9s 2

In [13]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk
